Let the input string consist of n letters, a1... an.
Let the grammar contain r terminal and nonterminal symbols R1... Rr. 
This grammar contains the subset Rs which is the set of start symbols.

Let P[n,n,r] be an array of booleans. Initialize all elements of P to false. #dimensió de la matriu, amb tot a false

For each i = 1 to n
  For each unit production Rj → ai, set P[i,1,j] = true. #si la gramàtica pot generar els terminals, posar true

For each i = 2 to n -- Length of string
  For each j = 1 to n-i+1 -- Start of string
    For each k = 1 to i-1 -- Partition of string
      For each production RA -> RB RC
        If P[j,k,B] and P[j+k,i-k,C] then set P[j,i,A] = true


If any of P[1,n,x] is true (x is iterated over the set s, where s are all the indices for Rs)
  Then string is member of language
  Else string is not member of language

S → a | XA | AX | b
A → RB
B → AX | b | a
X → a
R → XB

heads ['S', 'A', 'B', 'X', 'R'], bodies [['a', 'XA', 'AX', 'b'], ['RB'], ['AX', 'b', 'a'], ['a'], ['XB']]

In [50]:
var = 'XAD'
var[1:]

'AD'

In [78]:
current_body = ['a', 'ε']
current_body.remove('ε')
current_body

['a']

In [175]:
import string

grammar = 'grammar1.txt'
with open(grammar, 'r') as file:
    # Read the contents of the file and save them as a class variable
    contents = file.read()

def replace_letter(bodies, letter_to_combine):
    result = []
    for body in bodies:
        body = [letter for letter in body]
        if all([letter!=letter_to_combine for letter in body]):
            permutations = [''.join(body)]
        elif body != [letter_to_combine,letter_to_combine]:           
            permutations = []
            for combination in [letter_to_combine,None]:
                new_values = []
                for value in body:
                    if value == letter_to_combine: # If value is A (needs to be replaced)
                        if combination != None: # If it is not empty string, subtitute A with body
                            new_values.append(combination)   
                    else: # If it is not A, add value
                        new_values.append(value)
                permutations.append(''.join(new_values))
            else:
                if combination != None:
                    permutations.append(''.join(body))
        else:
            permutations = [letter_to_combine+letter_to_combine, letter_to_combine]
        result += permutations
    return result

def convert_grammar(grammar, cnf = True):
    
    # Split the grammar string into separate lines
    grammar_lines = grammar.strip().split('\n')
    heads, bodies = [], []
    for rule_idx in range(len(grammar_lines)): # Process each line of the grammar
        line = grammar_lines[rule_idx]
        head, elements = line.split(' → ') # Splitting the line into the head and elements of the rule
        values = elements.split(' | ') # Splitting the elements using "|" as the separator, for the cases of 'OR' statements
        heads.append(head)
        bodies.append(values)

    print(heads)
    print(bodies)
    
    if cnf:
        alphabet = [letter for letter in string.ascii_uppercase if letter not in heads]
        i = 0
        while i < len(bodies):
            for element in range(len(bodies[i])):
                current_var = bodies[i][element]
                #step 1:
                ''' 
                Eliminate terminals from right side if they exist with other terminals or non-terminals
                e.g: production rule X->xY can be decomposed as:
                X->ZY
                Z->x
                '''
                if any(char.islower() for char in current_var) and len(current_var) > 1:
                    new_var = ''
                    for letter in range(len(current_var)):
                        if current_var[letter].islower():
                            new_letter = alphabet.pop(0)
                            heads.append(new_letter)
                            bodies.append([current_var[letter]])
                            new_var += new_letter
                        else:
                            new_var += current_var[letter]
                    bodies[i][element] = new_var
                
                current_var = bodies[i][element] #in case something has changed
                
                #step 2:
                '''
                Eliminate RHS with more than two non-terminals.
                e.g,; production rule X->LYZ can be decomposed as:
                X->LP
                P->YZ
                '''
                if len(current_var) > 2:
                    new_letter = alphabet.pop(0)
                    new_var = current_var[0] + new_letter #Change to two terminals
                    bodies[i][element] = new_var #change the body of the rule to a two terminals rule
                    heads.append(new_letter) #add new created rule to heads
                    bodies.append([current_var[1:]]) #add new rule (the rest of the body of the original one) to bodies 
                
                current_var = bodies[i][element] #in case something has changed    
            i+=1


        #step 3:
        '''
        Eliminate ε-rules
        e.g: a production rule can derive is nullable in two scenarios:
            - ε appears explicitly: A → ε or  A → a | ε 
            - the rule is implicitly nullable: A → B where B doesn't exist
        For purely nullable rules: all istances of the head are deleted
        For partially nullable rules: consider that all appearances of the rule as possible empty string 
        '''
        #Dealing with implicitly nullable rules A → B
        for body in range(len(bodies)):
            current_body = bodies[body]
            if 'ε' in current_body or ((len(current_body) == 1) and (current_body[0].isupper() and (len(current_body[0])==1)) ): # Rule is nullable
                if len(current_body) == 1 and current_body != 'ε': 
                    current_body == 'ε'
                    bodies[body][element] = 'ε' #we delete A → B
        
        #dealing with explicitly pure nullable rule A → ε
        for body in range(len(bodies)): 
            current_body = bodies[body]
            if 'ε' in current_body and len(current_body)==1:
                to_delete = heads[body] #Rule that should be deleted from other rules
                for rule in range(len(bodies)):
                    if bodies[rule]:
                        updated_rule = [element.replace(to_delete, '') for element in bodies[rule]] #delete all the heads appearences in each rule
                        bodies[rule] = updated_rule
                heads[body] = False
                bodies[body] = False
        
        #updating lists to delete the null rules
        bodies = [element for element in bodies if element != False]
        heads = [element for element in heads if element != False]

        #dealing with explicitly impure nullable rule A → a | ε 
        for body in range(len(bodies)):
            current_body = bodies[body]
            if len(current_body) > 1 and 'ε' in current_body: 
                current_body.remove('ε') #elements for the permutation
                bodies[body] = current_body #we update the rule so it doesn't have ε 
                #modify the production rules:
                null_head = heads[body]
                nulled_bodies = [] #list of bodies that contain the nullable rule
                for body2 in range(len(bodies)): #we find the heads of the elements that contain the nullable rule
                    for element in range(len(bodies[body2])):
                        if null_head in bodies[body2][element]: #if the nullable rule appears in some other rule
                            nulled_bodies.append(heads[body2]) 
                for nulled in nulled_bodies: #3.1 i 3.2
                    nulled_index = heads.index(nulled)
                    new_bodies = replace_letter(bodies[nulled_index], null_head)
                    bodies[nulled_index] = new_bodies
                    changed_rule = heads[nulled_index]
                    for body3 in range(len(bodies)):
                        if any(changed_rule in el for el in bodies[body3]):
                            new_bodies = replace_letter(bodies[body3], changed_rule)
                            bodies[body3] = new_bodies

        #step 4:
        '''
        Eliminate unit rules
        e.g: considering production rule A → B:
            if B → CD exists, original rule should be converted to  A → CD
        '''
        for body in range(len(bodies)):
            current_body = bodies[body]
            if ((len(current_body) == 1) and (current_body[0].isupper() and (len(current_body[0])==1)) ): # Rule is unitary
                if heads.index(bodies[body]):
                    actual_rule = bodies[heads.index(bodies[body])] #he look for B → CD, specifically CD
                    bodies[body] = actual_rule
        
        #step 5:
        '''
        Filter unitary rules that are left
        '''
        bodies = [[item for item in sublist if not (len(item) == 1 and item.isupper())] for sublist in bodies]


                       
             
    #encoding the rules: each rule name will be encoded with a number from 0 to the amount of rules-1
    encoding_dict, encoding = {}, 0 # Creating an empty dictionary to store the rules and initializing the encoding
    for line in range(len(heads)):
        encoding_dict[heads[line]] = encoding
        encoding += 1
            
    #Creating a list for each type of rule (terminal and non-terminal). The index states the head code and the content its body
    ter_rules, nter_rules = [[] for _ in range(encoding)], [[] for _ in range(encoding)]
    for head,body in zip(heads, bodies): # Process each line of the grammar
        for element in body: 
            if element.isupper(): #it's a non-terminal
                #Creating a tuple so that 'AB' ~ ('A','B'), for future implementation use
                tuple_form = (encoding_dict[element[0]],encoding_dict[element[1]]) 
                nter_rules[encoding_dict[head]].append(tuple_form)
            else: #is a terminal
                ter_rules[encoding_dict[head]].append(element)

    return ter_rules, nter_rules

t_rules, nt_rules = convert_grammar(contents)


['S', 'A', 'B', 'X', 'R']
[['a', 'XA', 'AX', 'b'], ['RB'], ['AX', 'b', 'a'], ['a'], ['XB']]


In [158]:
grammar = 'grammar1.txt'
with open(grammar, 'r') as file:
    # Read the contents of the file and save them as a class variable
    contents = file.read()



def convert_grammar(grammar, cnf = True):
    
    # Split the grammar string into separate lines
    grammar_lines = grammar.strip().split('\n')
    
    rules = [[] for _ in range(len(grammar_lines))] #original length is the amount of rules
    
    #encoding the rules: each rule name will be encoded with a number from 0 to the amount of rules-1
    encoding_dict, encoding = {}, 0 # Creating an empty dictionary to store the rules and initializing the encoding
    for line in range(len(grammar_lines)):
        encoding_dict[grammar_lines[line][0]] = encoding
        encoding += 1

    #Creating a list for each type of rule (terminal and non-terminal). The index states the head code and the content its body
    t_rules, nt_rules = [[] for _ in range(encoding)], [[] for _ in range(encoding)]
    
    for line in grammar_lines: # Process each line of the grammar
        head, elements = line.split(' → ') # Splitting the line into the head and elements of the rule
        values = elements.split(' | ') # Splitting the elements using "|" as the separator, for the cases of 'OR' statements
        for element in values: 
            if len(element) > 2: #The rule is not found in Chomsky Natural Form
                rules_to_create = len(element)-1 #we have to create some we rules, 1 less than tha amount of non-terminals
                encoding_dict['X'+encoding] = encoding

            if element.isupper(): #it's a non-terminal
                #Creating a tuple so that 'AB' ~ ('A','B'), for future implementation use
                tuple_form = (encoding_dict[element[0]],encoding_dict[element[1]]) 
                nt_rules[encoding_dict[head]].append(tuple_form)
            else: #is a terminal
                t_rules[encoding_dict[head]].append(element)
    print(t_rules)

convert_grammar(contents)


[[], [], ['b'], ['a', 'b'], ['d', 'ε']]


In [6]:
grammar = 'grammar1.txt'
with open(grammar, 'r') as file:
    # Read the contents of the file and save them as a class variable
    contents = file.read()
contents

'\nS → a | XA | AX | b\nA → RB\nB → AX | b | a\nX → a\nR → XB'

In [ ]:
def CNF(grammar):
    
    pass

grammar = 'grammar1.txt'
with open(grammar, 'r') as file:
    # Read the contents of the file and save them as a class variable
    contents = file.read()
CNF(contents)

In [ ]:
from itertools import product

class Dynamic_CKY:
    
    def __init__(self, rules, grammar):
        self.grammar = grammar
        self.rules = rules
        self.amount_rules = 5

    def read_file(self):
        '''
        This function is used to read a '.txt' file containing the rules of a certain grammar.
        '''
        # Open the file in read mode using the file name
        with open(self.grammar, 'r') as file:
            # Read the contents of the file and save them as a class variable
            contents = file.read()
            return contents

    def convert_grammar(self):
        ''' 
        Converts the grammar rules into the format the class works with.
        Args:
            self.contents('.txt' file): grammar rules 
        Generates:
            encoding_dict(dictionary): Encoding used for the rules, where the key is the head of the rule and the body is the numerical encoding used.
            self.t_rules(list): Grammar rules, considering only the ones that derive terminals. 
            self.nt_rules(list): Grammar rules, considering only the ones that derive non-terminals.
        '''
        grammar = self.read_file() #Reading the grammar file
        # Split the grammar string into separate lines
        grammar_lines = grammar.strip().split('\n')
        #encoding the rules: each rule name will be encoded with a number from 0 to the amount of rules-1
        self.encoding_dict, encoding = {}, 0 # Creating an empty dictionary to store the rules and initializing the encoding
        for line in range(len(grammar_lines)):
            self.encoding_dict[grammar_lines[line][0]] = encoding
            encoding += 1
        #Creating a list for each type of rule (terminal and non-terminal). The index states the head code and the content its body
        self.t_rules, self.nt_rules = [[] for _ in range(encoding)], [[] for _ in range(encoding)]
        for line in grammar_lines: # Process each line of the grammar
            head, elements = line.split(' → ') # Splitting the line into the head and elements of the rule
            values = elements.split(' | ') # Splitting the elements using "|" as the separator, for the cases of 'OR' statements
            for element in values: 
                if element.isupper(): #it's a non-terminal
                    #Creating a tuple so that 'AB' ~ ('A','B'), for future implementation use
                    tuple_form = (self.encoding_dict[element[0]],self.encoding_dict[element[1]]) 
                    self.nt_rules[self.encoding_dict[head]].append(tuple_form)
                else: #is a terminal
                    self.t_rules[self.encoding_dict[head]].append(element)
    
    def test_word(self, word):
        ''' 
        Tests a certain word for the processed grammar.
        Input:
            word(string): The word to check by the CKY.
        Returns:
            boolean: True or False according to belonging to the grammar.
        Example:
            >>> cky = Dynamic_CKY(grammar)
            >>> cky.test_word('aabab')
            True
        '''
        self.word = word
        self.length = len(word)
        self.build_table()
        
    def build_table(self):
        '''
        Builds the table for the dynamic programming version of the CKY.
        Input:
            self.contents('.txt' file): Grammar rules.
            self.word(string): The word to check by the CKY. 
        Returns:
            self.table(list): Table with the derivation rules of the given word for the corresponding grammar.
        '''
        self.table = [[set() for p in range(self.length)]for i in range(self.length)]
        # We make sure the grammar can generate the terminals
        for terminal in range(self.length):
            self.table[self.length-1][terminal] = self.rules[word[terminal]]
        for i in range(self.length-2,-1, -1): # fila
            for j in range(0,i+1,1): # columna
                i1, j1 = self.length-1, j
                i2, j2 = i+1, j+1
                while i1 > i:
                    # Fer combinacions
                    for combi in self.combination((i1,j1),(i2,j2)):
                        if combi in rules:
                            if (rules[combi]) == 1: self.table[i][j].add(rules[combi]) 
                            else : self.table[i][j].update(rules[combi]) 
                    i1, i2, j2 = i1-1, i2+1, j2+1 
  
    def combination(self, cell1, cell2):
        return list(product(self.table[cell1[0]][cell1[1]], self.table[cell2[0]][cell2[1]]))
    

In [ ]:
grammar = 'grammar1.txt'
cky = Dynamic_CKY(grammar)

In [120]:
def replace_letter(bodies, letter_to_combine):
    result = []
    for body in bodies:
        body = [letter for letter in body]
        permutations = []
        for combination in [letter_to_combine,None]:
            print(combination)
            new_values = []
            amount = sum([letter == letter_to_combine for letter in body])
            for i in amount:
                for value in body:
                    if value == letter_to_combine: # If value is A (needs to be replaced)
                        if combination != None: # If it is not empty string, subtitute A with body
                            new_values.append(combination)   
                    else: # If it is not A, add value
                        new_values.append(value)
                permutations.append(''.join(new_values))
            else:
                if combination != None:
                    permutations.append(''.join(body))
        result += permutations
    return result
replace_letter(['DD'],'D')

D
None


['DD', '']